In [ ]:
import pandas as pd
import sys
from itertools import combinations

In [ ]:
sys.path.append('..\\packages')

In [ ]:
from suppression_check import DataAnonymizer

# Corrected relative path to the CSV file

In [ ]:
file_path = r'..\data\TwoSensitiveOverlapVerseLess2.csv'

# Load the CSV file into a DataFrame

In [ ]:
df_data = pd.read_csv(file_path)

In [ ]:
sensitive_list = ['Subgroup1', 'Subgroup2']

# Instantiate the anonymizer

In [ ]:
anonymizer = DataAnonymizer(df_data, sensitive_columns=sensitive_list, frequency='Counts', minimum_threshold=9)

In [ ]:
df_log = anonymizer.create_log()

In [ ]:
df_log.to_excel('1CreateLog.xlsx', index=False)

# Less Than Threshold 

In [ ]:
df_less_than_threshold = anonymizer.less_than_threshold_not_zero()

In [ ]:
df_less_than_threshold.to_excel('2LessThanThreshold.xlsx', index=False)

# Apply Redact Threshold

In [ ]:
df_less_than_threshold.loc[:, 'Overlapping'] = 0

# Loop through each sensitive column to check for overlapping sensitive information
for sensitive_column in sensitive_list:
    list_sensitive = df_less_than_threshold[df_less_than_threshold['RedactBinary'] == 1][sensitive_column].unique()
    mask = df_less_than_threshold[sensitive_column].isin(list_sensitive) & (df_less_than_threshold['RedactBinary'] != 1)
    df_less_than_threshold.loc[mask, 'Overlapping'] += 1
if max(list(df_less_than_threshold['Overlapping'].unique())) == 1:
    df_less_than_threshold = anonymizer.one_count_redacted()
    display(df_less_than_threshold)
    df_less_than_threshold.loc[:, 'Overlapping'] = 0
    for sensitive_column in sensitive_list:
        list_sensitive = df_less_than_threshold[df_less_than_threshold['RedactBinary'] == 1][sensitive_column].unique()
        mask = df_less_than_threshold[sensitive_column].isin(list_sensitive) & (df_less_than_threshold['RedactBinary'] != 1)
        df_less_than_threshold.loc[mask, 'Overlapping'] += 1
display(df_less_than_threshold)

In [ ]:
df_redact_threshold = anonymizer.redact_threshold()

In [ ]:
df_redact_threshold.to_excel('3RedactThreshold.xlsx', index=False)

In [ ]:
df_apply_log = anonymizer.apply_log()

In [ ]:
df_apply_log.to_excel('4ApplyLog.xlsx', index=False)